In [1]:
import numpy as np
from numba import njit
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs
import multiprocessing as mp
import os
import time

# --- 1. 加载我们在第一步生成的真值 ---
try:
    TRUE_GAMMAS = np.load("riemann_10k_true.npy")
    print(f">>> 成功加载权威真值，共 {len(TRUE_GAMMAS)} 个零点。")
except:
    print("!!! 错误：未找到 riemann_10k_true.npy，请先运行 gen_truth.py")
    exit()

# --- 2. 动力学内核 (带数值稳定保护) ---
@njit(fastmath=True, nogil=True)
def target_kernel(u_c, k, steps, n_bins):
    x = 0.5
    counts = np.zeros((n_bins, n_bins), dtype=np.float64)
    last_bin = 0
    
    # 200万步热启动，保证概率分布收敛
    for i in range(steps + 2000000):
        # 动态老化项：微观调节
        k_dynamic = k / (np.log(i + 100)**2)
        x = 1 - (u_c + k_dynamic) * x**2
        
        # 数值截断保护
        if x > 1.0: x = 0.999
        if x < -1.0: x = -0.999
            
        current_bin = int((x + 1.0) / 2.0 * (n_bins - 1))
        
        if i > 2000000:
            if 0 <= current_bin < n_bins:
                counts[last_bin, current_bin] += 1
                last_bin = current_bin
    return counts

# --- 3. 狙击手 Worker ---
def sniper_worker(task):
    # 任务包：(分段ID, 起始索引, 结束索引, 物理预测的k值)
    seg_idx, start_n, end_n, k_val = task
    
    U_C = 1.543689012
    N_BINS = 20000
    STEPS = 10**10 # 维持高精度
    
    # 🌟 修改点 1：新开辟一个纯净版目录，防止覆盖原有含4%误差的真实对照数据
    SAVE_DIR = "riemann_10k_harvest_pure"
    
    try:
        t0 = time.time()
        counts = target_kernel(U_C, k_val, STEPS, N_BINS)
        
        if np.sum(counts) < 1000:
            return f"Seg {seg_idx}: Matrix Empty (k={k_val:.4f})"

        P = csr_matrix(counts)
        row_sums = np.array(P.sum(axis=1)).flatten()
        row_sums[row_sums == 0] = 1.0
        P = P.multiply(1.0 / row_sums[:, np.newaxis])
        
        # 提取目标区间的特征值
        num_targets = end_n - start_n
        
        # 🌟 修改点 2：特征值索要量必须翻倍！因为接下来会扔掉一半的复共轭态
        calc_k = num_targets * 2 + 150 # 加大缓冲
        
        v0 = np.ones(N_BINS) # 初始向量防止 ARPACK error
        vals, _ = eigs(P, k=calc_k, which='LM', tol=1e-4, v0=v0)
        
        # 🌟 修改点 3：只取正能量态的物理过滤器！斩断 vals.imag < 0 的影子骨架
        pure_vals = vals[(np.abs(vals) > 0.4) & (vals.imag > 1e-8)]
        phases = np.sort(np.angle(pure_vals))
        
        # --- 核心：现场比对 (In-flight Validation) ---
        if len(phases) > 0:
            true_segment = TRUE_GAMMAS[start_n:end_n]
            
            # 简单锚定：用第一个点对齐
            scale = true_segment[0] / phases[0]
            sim_segment = phases[:len(true_segment)] * scale
            
            # 计算这一段的平均误差
            avg_err = np.mean(np.abs(sim_segment - true_segment[:len(sim_segment)]))
        else:
            avg_err = 999.9

        # 保存纯正能量相角数据
        if not os.path.exists(SAVE_DIR):
            os.makedirs(SAVE_DIR, exist_ok=True)
        filename = os.path.join(SAVE_DIR, f"seg_{seg_idx}_k_{k_val:.4f}_err_{avg_err:.2f}.npy")
        np.save(filename, phases)
        
        return f"Seg {seg_idx} [n={start_n+1}-{end_n}] | k={k_val:.4f} | Err={avg_err:.3f} | {time.time()-t0:.1f}s"
    except Exception as e:
        return f"Seg {seg_idx} ERR: {str(e)}"

# --- 4. 战役指挥部 ---
if __name__ == "__main__":
    # 🌟 同步修改主进程目录
    SAVE_DIR = "riemann_10k_harvest_pure"
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
        
    print(f">>> 启动‘万点长征’ (纯正能量版) 收割计划")
    print(f">>> 物理导航公式: k = 4.7 + 10.13/ln(n)")
    
    tasks = []
    total_points = 10000
    segment_size = 100 # 每 100 个点一组，方便并行
    
    for i in range(0, total_points, segment_size):
        # 索引转换：i=0 对应 n=1
        start_n = i
        end_n = min(i + segment_size, total_points)
        
        # 计算该段中心的 n 值（用于确定最佳 k）
        mid_n = (start_n + end_n) / 2 + 1 # +1 因为黎曼序号从1开始
        if mid_n < 2: mid_n = 2
        
        # 物理公式计算 k
        k_opt = 4.7000 + 10.13 / np.log(mid_n)
        
        tasks.append((i//segment_size, start_n, end_n, k_opt))
        
    print(f">>> 任务分发: {len(tasks)} 个分段，覆盖 {total_points} 个零点")
    
    # 100 并发，稳健模式
    BATCH_SIZE = 100
    total_batches = (len(tasks) + BATCH_SIZE - 1) // BATCH_SIZE
    
    for i in range(total_batches):
        batch_tasks = tasks[i*BATCH_SIZE : (i+1)*BATCH_SIZE]
        print(f"\n[Batch {i+1}/{total_batches}] 正在收割...")
        
        with mp.Pool(processes=BATCH_SIZE) as pool:
            results = pool.map(sniper_worker, batch_tasks)
            
        for res in results:
            print(f"  {res}")

    print("\n>>> 万点纯正能量收割完成！请前往 Phase Unwrapping 环节检验误差是否被抹平！")

>>> 成功加载权威真值，共 10000 个零点。
>>> 启动‘万点长征’ (纯正能量版) 收割计划
>>> 物理导航公式: k = 4.7 + 10.13/ln(n)
>>> 任务分发: 100 个分段，覆盖 10000 个零点

[Batch 1/1] 正在收割...
  Seg 0 [n=1-100] | k=7.2764 | Err=397.060 | 1326.8s
  Seg 1 [n=101-200] | k=6.7190 | Err=9047.878 | 1400.5s
  Seg 2 [n=201-300] | k=6.5333 | Err=16870.727 | 1421.1s
  Seg 3 [n=301-400] | k=6.4284 | Err=9132.088 | 1404.2s
  Seg 4 [n=401-500] | k=6.3575 | Err=19595.927 | 1409.4s
  Seg 5 [n=501-600] | k=6.3049 | Err=18862.434 | 1416.4s
  Seg 6 [n=601-700] | k=6.2636 | Err=27666.003 | 1303.5s
  Seg 7 [n=701-800] | k=6.2299 | Err=66299.033 | 1400.8s
  Seg 8 [n=801-900] | k=6.2015 | Err=110286.294 | 1415.9s
  Seg 9 [n=901-1000] | k=6.1772 | Err=44729.530 | 1393.2s
  Seg 10 [n=1001-1100] | k=6.1560 | Err=27048.417 | 1333.1s
  Seg 11 [n=1101-1200] | k=6.1372 | Err=231569.075 | 1043.5s
  Seg 12 [n=1201-1300] | k=6.1204 | Err=25481.313 | 1393.1s
  Seg 13 [n=1301-1400] | k=6.1053 | Err=29003.790 | 1389.1s
  Seg 14 [n=1401-1500] | k=6.0915 | Err=30430.580 | 130